In [14]:
%%bash
# File downloaded from https://drive.google.com/drive/folders/1doez2I1G2NL2_kvu2n2PswZxQRp158Rb
progressiveMauve --output=test.xmfa GCA_000669655.1_Myco_tube_KZN_4207_V1_genomic.fna GCA_000669675.1_Myco_tube_KZN_1435_V1_genomic.fna

Storing raw sequence at /tmp/rawseq20006.000
Sequence loaded successfully.
references/GCA_000669655.1_Myco_tube_KZN_4207_V1_genomic.fna 4395092 base pairs.
Storing raw sequence at /tmp/rawseq20006.001
Sequence loaded successfully.
references/GCA_000669675.1_Myco_tube_KZN_1435_V1_genomic.fna 4401837 base pairs.
Using weight 15 mers for initial seeds
Sorted mer list loaded successfully
Sorted mer list loaded successfully
0%..1%..2%..3%..4%..5%..6%..7%..8%..9%..10%..
11%..12%..13%..14%..15%..16%..17%..18%..19%..20%..
21%..22%..23%..24%..25%..26%..27%..28%..29%..30%..
31%..32%..33%..34%..35%..36%..37%..38%..39%..40%..
41%..42%..43%..44%..45%..46%..47%..48%..49%..50%..
51%..52%..53%..54%..55%..56%..57%..58%..59%..60%..
61%..62%..63%..64%..65%..66%..67%..68%..69%..70%..
71%..72%..73%..74%..75%..76%..77%..78%..79%..80%..
81%..82%..83%..84%..85%..86%..87%..88%..89%..90%..
91%..92%..93%..94%..95%..96%..97%..98%..99%..done.
using default bp penalty: 154480
using default bp estimate min score: 46

In [ ]:
from Bio import SeqIO
import pandas as pd

In [30]:
# Extracting Alignemnt information

alignments = {"gid":[], 'start':[], 'end':[],'strand':[], 'file':[]}

for rec in SeqIO.parse('./test.xmfa', "fasta"):
    coord, strand, fl = rec.description.split()
    gid, start_end = coord.split(":")
    start, end = start_end.split("-")
    start, end = int(start)-1, int(end)
    alignments["gid"].append(gid)
    alignments["start"].append(start)
    alignments["end"].append(end)
    alignments["strand"].append(strand)
    alignments["file"].append(fl)
alignments = pd.DataFrame(alignments)
alignments = alignments.head(alignments.shape[0]//2 - 1) ## This part might not work for other samples
    

In [67]:
files = alignments.groupby("gid")["file"].apply(set).reset_index()
files["file"] = files["file"].apply(lambda x: list(x)[0]) 

In [70]:
sequences = {}
for _, row in files.iterrows():
    sequences[row["gid"]]={}
    for rec in SeqIO.parse(row["file"],"fasta"):
        sequences[row["gid"]]["sd"] = rec.id
        sequences[row["gid"]]["sz"] = len(rec.seq)
        sequences[row["gid"]]["sq"] = rec.seq
        

In [79]:
# Ordering the framents

first_seq = ""
second_seq = ""
pos1 = alignments.loc[alignments.gid=="1", "start"].tolist() +alignments.loc[alignments.gid=="1", "end"].tolist()
pos2 = alignments.loc[alignments.gid=="2", "start"].tolist() +alignments.loc[alignments.gid=="2", "end"].tolist()
pos1.sort()
pos2.sort()

for i in range(0,len(alignments),2):
    if pos1[-1] ==alignments.loc[i, 'end']:
        tseq1 = sequences[alignments.loc[i, 'gid']]["sq"][alignments.loc[i, 'start']:]+sequences[alignments.loc[i, 'gid']]["sq"][:min(pos1)]
    else:
        tseq1= sequences[alignments.loc[i, 'gid']]["sq"][alignments.loc[i, 'start']: pos1[pos1.index(alignments.loc[i, 'end'])+1]]

    if pos2[-1] ==alignments.loc[i+1, 'end']:
        tseq2= sequences[alignments.loc[i+1, 'gid']]["sq"][alignments.loc[i+1, 'start']:]+sequences[alignments.loc[i+1, 'gid']]["sq"][:min(pos2)]
    else:
        tseq2= sequences[alignments.loc[i+1, 'gid']]["sq"][alignments.loc[i+1, 'start']: pos2[pos2.index(alignments.loc[i+1, 'end'])+1]]
    if alignments.loc[i, 'strand']=="-":
        tseq1 = tseq1.reverse_complement()
    if alignments.loc[i+1, 'strand'] =="-":
        tseq2 = tseq2.reverse_complement()
    first_seq += tseq1
    second_seq += tseq2
    
with open("serialised_and_inversed.fasta","w") as fout:
    fout.write(f">{sequences['1']['sd']}\n{first_seq}\n>{sequences['2']['sd']}\n{second_seq}")

In [80]:
%%bash
mafft --auto  serialised_and_inversed.fasta > serialised_and_inversed_aln.fasta

nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..

There are 8349 ambiguous characters.
    1 / 2
done.

Constructing a UPGMA tree (efffree=1) ... 
    0 / 2
done.

Progressive alignment 1/1... 
STEP     1 / 1 
len1=4395092, len2=4401837, Switching to the memsave mode
m
Many anchors were found. The upper-level DP is skipped.

DP 00001 / 28868DP 00002 / 28868DP 00003 / 28868DP 00004 / 28868DP 00005 / 28868DP 00006 / 28868DP 00007 / 28868DP 00008 / 28868DP 00009 / 28868DP 00010 / 28868DP 00011 / 28868DP 00012 / 28868DP 00013 / 28868DP 00014 / 28868DP 00015 / 28868DP 00016 / 28868DP 00017 / 28868DP 00018 / 28868DP 00019 / 28868DP 00020 / 28868DP 00021 / 28868DP 00022 / 28868DP 00023 / 28868DP 00024 / 28868DP 00025 / 28868DP 00026 / 28868DP 00027 / 28868DP 00028 / 28868DP 00029 / 28868DP 00030 / 28868DP 00031 / 28868DP 00032 / 28868DP 00

In [ ]:
%% bash
vg construct -M serialised_and_inversed_aln.fasta -t 2 > serialised_and_inversed_aln.vg

In [ ]:
%%bash
vg view -d serialised_and_inversed_aln.vg > serialised_and_inversed_aln.dot

In [ ]:
%% bash
dot -Tsvg serialised_and_inversed_aln.dot -o res.svg